# Final Generation of Sampling Points

This function is where we take collection of sampling points which are found from emissions data, and we perfrom:
1.  Fit the data, remove values from the set which are below a threshold length and $R^2$ value
2.  Fit with least squares regression, forming sampling points which follow the line/ our best estimate of where it is, and parallel ship tracks along the path
3.  Rejoin the data set together, with actual points, fitted points, and counterfactuals sorted and grouped by particle number. 
4. Run for every day in 2019.

In [1]:
import numpy as np
import pandas as pd
import pathlib
from datetime import datetime, timedelta, time
import matplotlib.pyplot as plt
import warnings
import calendar
import time
warnings.filterwarnings("ignore")
R_Squared = 0.97

In [2]:
def DataCleaningAndAssignment(df, date): #This will take an input of a dataset, which we must save as df. 
    SortedParticles = df.sort_values(by = "particle")
    grouped_data = SortedParticles.groupby('particle')
    subsets = [group for _, group in grouped_data]
    #We now complete the data cleaning. This involves removing all tracks of length less than 10, and all which do not meet a minimum R**2 value. 
    for subset in subsets:
        subset.dropna(inplace=True) #removing any non numerical values
        if len(subset) >= 10:
            Xvar = subset['longitude'].values
            Yvar = subset['latitude'].values
            if len(Xvar) == 0:
                continue
            try: 
                coefficients = np.polyfit(Xvar, Yvar, 2)
# Calculate the predicted values using the polynomial fit
                y_pred = np.polyval(coefficients, Xvar)
# Calculate the correlation coefficient (R-squared)
                correlation_matrix = np.corrcoef(Yvar, y_pred)
                r_squared = correlation_matrix[0, 1] ** 2  # Squared to get R-squared
                subset['R**2 value'] = r_squared 
                subset['X =?'] = 'Long' 
            except Exception as e:
            # Handle the LinAlgError here
                print(f"LinAlgError: {e}. Skipping subset {subset['particle'].iloc[0]}.")
                subset['R**2 value'] = 0  
                pass
            if r_squared < R_Squared:                 #If the line is of a low quality, then we rotate and see if y vs x gives better data. 
                Xvar = subset['latitude'].values
                Yvar = subset['longitude'].values
                if len(Xvar) == 0:
                    continue
                coefficients = np.polyfit(Xvar, Yvar, 2)
# Calculate the predicted values using the polynomial fit
                y_pred = np.polyval(coefficients, Xvar)
# Calculate the correlation coefficient (R-squared)
                correlation_matrix = np.corrcoef(Yvar, y_pred)
                r_squared = correlation_matrix[0, 1] ** 2  # Squared to get R-squared
                subset['R**2 value'] = r_squared 
                if r_squared > 0.97:
                    subset['X =?'] = 'Lat'  
                else:
                    subset['X =?'] = 'neither' 

        else:
            subset['R**2 value'] = 0
            continue
# This is where box 1 becomes box 2 in the origional script            
# 
    Spacing = 0.27
    for subset in subsets:
        subset.sort_values(by='hour', inplace=True)
        if subset.iloc[1][-2] >= 0.97 and subset.iloc[0][-1] == 'Long':
            Xvar = subset['longitude'].values
            Yvar = subset['latitude'].values
            coefficients = np.polyfit(Xvar, Yvar, 2)
            Xplotted = np.linspace(Xvar.min(), Xvar.max(), len(subset))
            Yplotted = []
            for i in range(len(subset)):
                Yplotted.append(coefficients[0]*Xplotted[i]**2 + coefficients[1]*Xplotted[i] + coefficients[2])
            if subset.iloc[0]['longitude'] <= subset.iloc[-1]['longitude']: #If the ship is moving eastwards (i.e.if last entry in long is greater than first)
                subset['X Plotted Values'] = Xplotted
                subset['Y Plotted Values'] = Yplotted
            else:
                subset['X Plotted Values'] = Xplotted[::-1]
                subset['Y Plotted Values'] = Yplotted[::-1]           
    #here we are now using our previous function
            SlopeVector = []
            for i in range(len(Xplotted)-1):
                SlopeVector.append((Yplotted[i+1]-Yplotted[i])/(Xplotted[i+1]-Xplotted[i]))
            SlopeVector.append(SlopeVector[-1]) #we imagine to find the last point that the ship continues on its last heading. 
    #Given the slope, we now want to move outwards orthogonally to this. The direction of step at each point is given by m
            m = []
            for i in range(len(SlopeVector)):
                m.append(-SlopeVector[i]**-1)  #Taking advantage of simple euclidean geometry
            Upper_Long = []
            Upper_Lat  = []
            Lower_Long = []
            Lower_Lat  = []
            for i in range(len(Xplotted)):
                Upper_Long.append(Xplotted[i] + Spacing*(1 / np.sqrt(1 + m[i]**2)))
                Upper_Lat.append(Yplotted[i] + Spacing*m[i] / np.sqrt(1 + m[i]**2))
                Lower_Long.append(Xplotted[i] - Spacing*(1 / np.sqrt(1 + m[i]**2)))
                Lower_Lat.append(Yplotted[i] - Spacing*m[i] / np.sqrt(1 + m[i]**2))
    
            if subset.iloc[0]['longitude'] <= subset.iloc[-1]['longitude']: #i.e., if the ship is moving east (right)
                subset['X Imaginary Upper'] = Upper_Long
                subset['Y Imaginary Upper'] = Upper_Lat
                subset['X Imaginary Lower'] = Lower_Long
                subset['Y Imaginary Lower'] = Lower_Lat     
            else:
                subset['X Imaginary Upper'] = Upper_Long[::-1]
                subset['Y Imaginary Upper'] = Upper_Lat[::-1]
                subset['X Imaginary Lower'] = Lower_Long[::-1]
                subset['Y Imaginary Lower'] = Lower_Lat[::-1]
            
            #    subset['X Plotted Values'] = Xplotted[::-1]
             #   subset['Y Plotted Values'] = Yplotted[::-1]           
    #here we are now using our previous function
            SlopeVector = []
            for i in range(len(Xplotted)-1):
                SlopeVector.append((Yplotted[i+1]-Yplotted[i])/(Xplotted[i+1]-Xplotted[i]))
            SlopeVector.append(SlopeVector[-1]) #we imagine to find the last point that the ship continues on its last heading. 
    #Given the slope, we now want to move outwards orthogonally to this. The direction of step at each point is given by m
            m = []
            for i in range(len(SlopeVector)):
                m.append(-SlopeVector[i]**-1)  #Taking advantage of simple euclidean geometry
            Upper_Long = []
            Upper_Lat  = []
            Lower_Long = []
            Lower_Lat  = []
            for i in range(len(Xplotted)):
                Upper_Long.append(Xplotted[i] + Spacing*(1 / np.sqrt(1 + m[i]**2)))
                Upper_Lat.append(Yplotted[i] + Spacing*m[i] / np.sqrt(1 + m[i]**2))
                Lower_Long.append(Xplotted[i] - Spacing*(1 / np.sqrt(1 + m[i]**2)))
                Lower_Lat.append(Yplotted[i] - Spacing*m[i] / np.sqrt(1 + m[i]**2))
    
            if subset.iloc[0]['longitude'] <= subset.iloc[-1]['longitude']: #i.e., if the ship is moving east (right)
                subset['X Imaginary Upper'] = Upper_Long
                subset['Y Imaginary Upper'] = Upper_Lat
                subset['X Imaginary Lower'] = Lower_Long
                subset['Y Imaginary Lower'] = Lower_Lat     
            else:
                subset['X Imaginary Upper'] = Upper_Long[::-1]
                subset['Y Imaginary Upper'] = Upper_Lat[::-1]
                subset['X Imaginary Lower'] = Lower_Long[::-1]
                subset['Y Imaginary Lower'] = Lower_Lat[::-1]
            
        elif subset.iloc[1][-2] >= 0.97 and subset.iloc[0][-1] == 'Lat':
            Xvar = subset['latitude'].values
            Yvar = subset['longitude'].values
            coefficients = np.polyfit(Xvar, Yvar, 2)
            Xplotted = np.linspace(Xvar.min(), Xvar.max(), len(subset))
            Yplotted = []
            for i in range(len(subset)):
                Yplotted.append(coefficients[0]*Xplotted[i]**2 + coefficients[1]*Xplotted[i] + coefficients[2])
            if Xvar[0] <= Xvar[-1]: #If the ship is moving north
                subset['Y Plotted Values'] = Xplotted
                subset['X Plotted Values'] = Yplotted
            else:
                subset['Y Plotted Values'] = Xplotted[::-1]
                subset['X Plotted Values'] = Yplotted[::-1]           
    #here we are now using our previous function
            SlopeVector = []
            for i in range(len(Xplotted)-1):
                SlopeVector.append((Yplotted[i+1]-Yplotted[i])/(Xplotted[i+1]-Xplotted[i]))
            SlopeVector.append(SlopeVector[-1]) #we imagine to find the last point that the ship continues on its last heading. 
    #Given the slope, we now want to move outwards orthogonally to this. The direction of step at each point is given by m
            m = []
            for i in range(len(SlopeVector)):
                m.append(-SlopeVector[i]**-1)  #Taking advantage of simple euclidean geometry
            Upper_Long = []
            Upper_Lat  = []
            Lower_Long = []
            Lower_Lat  = []
            for i in range(len(Xplotted)):
                Upper_Long.append(Xplotted[i] + Spacing*(1 / np.sqrt(1 + m[i]**2)))
                Upper_Lat.append(Yplotted[i] + Spacing*m[i] / np.sqrt(1 + m[i]**2))
                Lower_Long.append(Xplotted[i] - Spacing*(1 / np.sqrt(1 + m[i]**2)))
                Lower_Lat.append(Yplotted[i] - Spacing*m[i] / np.sqrt(1 + m[i]**2))
    
            if Xvar[0] <= Xvar[-1]: #i.e., if the ship is moving east (right)
                subset['Y Imaginary Upper'] = Upper_Long
                subset['X Imaginary Upper'] = Upper_Lat
                subset['Y Imaginary Lower'] = Lower_Long
                subset['X Imaginary Lower'] = Lower_Lat     
            else:
                subset['Y Imaginary Upper'] = Upper_Long[::-1]
                subset['X Imaginary Upper'] = Upper_Lat[::-1]
                subset['Y Imaginary Lower'] = Lower_Long[::-1]
                subset['X Imaginary Lower'] = Lower_Lat[::-1]

    RealValues        = []
    FittedValues      = []
    UpperCounterFacs  = []
    LowerCounterFacs = []
    for subset in subsets:
        if subset.iloc[0][8] > 0.97:
            for i in range(len(subset)):
                RealValues.append([subset.iloc[i][0], subset.iloc[i][1], subset.iloc[i][2], subset.iloc[i][3], subset.iloc[i][4], subset.iloc[i][5], subset.iloc[i][6], subset.iloc[i][7], subset.iloc[i][8], 'Real LongLat']) 
            for j in range(len(subset)):
                FittedValues.append([subset.iloc[j][0], subset.iloc[j][1], subset.iloc[j][2], subset.iloc[j][3], subset.iloc[j][4], subset.iloc[j][5], subset.iloc[j]['Y Plotted Values'], subset.iloc[j]['X Plotted Values'], subset.iloc[j][8], 'LongLatFitted'])
            for k in range(len(subset)):
                UpperCounterFacs.append([subset.iloc[k][0], subset.iloc[k][1], subset.iloc[k][2], subset.iloc[k][3], subset.iloc[k][4], subset.iloc[k][5], subset.iloc[k]['Y Imaginary Upper'], subset.iloc[k]['X Imaginary Upper'], subset.iloc[k][8], 'UpperCounterfactual'])
            for l in range(len(subset)):
                LowerCounterFacs.append([subset.iloc[l][0], subset.iloc[l][1], subset.iloc[l][2], subset.iloc[l][3], subset.iloc[l][4], subset.iloc[l][5], subset.iloc[l]['Y Imaginary Lower'], subset.iloc[l]['X Imaginary Lower'], subset.iloc[l][8], 'LowerCounterfactual'])
        else:
            continue
            
    column = ['hour', 'y', 'x', 'mass', 'signal', 'particle', 'latitude', 'longitude', 'R**2 value', 'PointLabel']
    
    RealData = pd.DataFrame(RealValues, columns = column)
    FittedData = pd.DataFrame(FittedValues, columns = column) 
    UpperCounterFacData = pd.DataFrame(UpperCounterFacs, columns = column)
    LowerCounterFacData = pd.DataFrame(LowerCounterFacs, columns = column)
    
    RealData.to_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/RealData/{}_{}_{}'.format(str(date.year), str(date.month).zfill(2), str(date.day).zfill(2)))
    FittedData.to_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/FittedData/{}_{}_{}'.format(str(date.year), str(date.month).zfill(2), str(date.day).zfill(2)))
    UpperCounterFacData.to_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/UpperCounterFacData/{}_{}_{}'.format(str(date.year), str(date.month).zfill(2), str(date.day).zfill(2)))
    LowerCounterFacData.to_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/LowerCounterFacData/{}_{}_{}'.format(str(date.year), str(date.month).zfill(2), str(date.day).zfill(2)))

In [4]:
emissions_path = pathlib.Path("/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/emissions_tracked")
start_date = datetime(2019, 9, 13)  # day, month, year
end_date = datetime(2019, 12, 31)  # Adjust the end date as needed
current_date = start_date

while current_date <= end_date:
    file_path = emissions_path / f"{current_date.year:04d}" / f"{current_date.month:02d}_{current_date.day:02d}"
    if file_path.exists():
        Link = pd.read_csv(file_path)
        DataCleaningAndAssignment(Link, current_date)
        
        print(f"Processed data for {current_date}")
    else:
        print(f"No data found for {current_date}")

    # Move to the next day
    current_date += timedelta(days=1)

Processed data for 2019-09-13 00:00:00
Processed data for 2019-09-14 00:00:00
Processed data for 2019-09-15 00:00:00
Processed data for 2019-09-16 00:00:00
Processed data for 2019-09-17 00:00:00
Processed data for 2019-09-18 00:00:00
Processed data for 2019-09-19 00:00:00
Processed data for 2019-09-20 00:00:00
Processed data for 2019-09-21 00:00:00
Processed data for 2019-09-22 00:00:00
Processed data for 2019-09-23 00:00:00
Processed data for 2019-09-24 00:00:00
Processed data for 2019-09-25 00:00:00
Processed data for 2019-09-26 00:00:00
Processed data for 2019-09-27 00:00:00
Processed data for 2019-09-28 00:00:00
Processed data for 2019-09-29 00:00:00
Processed data for 2019-09-30 00:00:00
Processed data for 2019-10-01 00:00:00
Processed data for 2019-10-02 00:00:00
Processed data for 2019-10-03 00:00:00
Processed data for 2019-10-04 00:00:00
Processed data for 2019-10-05 00:00:00
Processed data for 2019-10-06 00:00:00
Processed data for 2019-10-07 00:00:00
Processed data for 2019-1